# 七、Python的高级特性

Python的高级特性主要包括迭代器与生成器、装饰器、描述器、上下文管理器和猴子补丁。

## 迭代器与生成器

基本上， 迭代器是含有 .next() 方法的对象。在Python中循环枚举一个对象时，其实在调用对象中调用iter（）方法。该函数返回一个定义了next（）方法的迭代器对象，它在容器中逐一访问元素。没有后续的元素时， next（）抛出一个 StopIteration异常。使用Python内置的iter（）方法可以生成一个迭代器。

In [1]:
a = iter([0, 1, 2])

In [2]:
a

In [3]:
next(a)

0

In [4]:
next(a)

1

In [5]:
next(a)

2

In [6]:
next(a)

StopIteration: 

在for循环中就是由这个StopIteration异常通知for语句循环结束的。

生成器是创建迭代器的简单而强大的工具。它基于yield指令，允许停止函数并立即返回结果。
因为其保存其执行上下文，所以还有以下优点：
1. 惰性求值； 
2. 在处理大的列表时不需要一次性加载全部数据，可以减小内存使用； 
除非特殊的原因，应该在代码中使用生成器。

In [7]:
# 定义一个生成器
def my_range(n):
    i = 0
    while i < n:
        yield i
        i += 1

In [8]:
a = my_range(3)

In [9]:
a

<generator object my_range at 0x7fae1f31a960>

In [10]:
a.next()

0

In [11]:
a.next()

1

In [12]:
a.next()

2

In [13]:
a.next()

StopIteration: 

除了使用yield之外，还可以使用生成器表达式来产生一个生成器。如

In [14]:
a = (i for i in range(3))

In [15]:
a

<generator object <genexpr> at 0x7fae1f31a9b0>

注意，这里使用圆括号而非方括号。

## 装饰器

装饰器的作用是在原有对象的基础上添加额外功能。python中可以将函数作为参 数，进行装饰，返回经过修饰过的函数，比如：

In [16]:
def makebold(fn):
    def wrapper():
        return "<b>" + fn() + "</b>"
    return wrapper

def hello_world():
    return "hello world"

In [17]:
# 使用装饰器前
hello_world()

'hello world'

In [18]:
# 使用装饰器后
f = makebold(hello_world)
f()

'<b>hello world</b>'

Python还可以使用语法糖@来达到相同的作用。

In [19]:
@makebold
def hello_world():
    return "hello world"

In [20]:
hello_world()

'<b>hello world</b>'

## 描述器

在python中一个描述器就是定义下面的方法中一个或多个的一个对象：

In [ ]:
__get__(self, instance, owner)
__set__(self, instance, value)
__delete__(self, instance)

如果一个对象同时定义了`__get__()`和`__set__()`，它叫做资料描述器。只定义了`__get__()`的描述器叫做非资料描述器(一般用于方法)。 资料描述器和非资料描述器的区别在于：相对于实例字典的优先级。如果实例字典中有与资料描述器同名的属性，优先使用资料描述器中的；如果实例字典中有与非资料描述器中同名的属性，优先使用实例字典中的。即优先级 资料描述器 > 实例字典 > 非资料描述器。 
要想写一个只读的资料描述器，只需同时定义`__get__()`和`__set__()`并在`__set__()`中抛出一个AttributeError。

Python中包括函数，属性, 静态方法, 类方法这些技术都是基于描述器。

## 上下文管理器

上下文管理器是Python2.5开始支持的一种语法，用于处理指 定代码块进入和退出时的操作。一般使用with语法，也可以直接调用相应的方法。

with语句是用来简化“try/finally”语句的，通常用于处理共享资源的获取和 释放，比如文件、数据库和线程资源。比如：

In [ ]:
with VAR = EXPR:
    BLOCK

相当于进行了如下操作：

In [ ]:
VAR = EXPR
VAR.__enter__()
try:
    BLOCK
finally:
    VAR.__exit__()

下面通过一个实际的例子来展示上下文管理器的创建和使用。

In [21]:
import time
class timeit:
    def __init__(self,label):
        self.label = label

    def __enter__(self):
        self.start = time.time()

    def __exit__(self, exc_ty, exc_val, exc_tb):
        end = time.time()
        print ('{}: {}'.format(self.label, end - self.start))

In [22]:
with timeit('counting'):
    n = 10000
    while n > 0:
        n -= 1;

counting: 0.00205016136169


上述也可利用@contextmanager装饰器来简化代码。

In [23]:
from contextlib import contextmanager
import time

@contextmanager
def timeit(label):
    start = time.time()
    try:
        yield
    finally:
        end = time.time()
        print('{}: {}'.format(label, end - start))

In [24]:
with timeit('counting'):
    n = 10000
    while n > 0:
        n -= 1;

counting: 0.00238680839539


## 猴子补丁

猴子补丁是用来在运行时动态修改已有的代码。它可以不去改变源码而对功能进行追加和变更。

比如在前面类的章节中，父类的job是工人。现在晋升为管理层了，这时不需要修改原代码，只要加个补丁就搞定。

In [25]:
class Parent():
    name = "tom"
        
    def job(self):
        print("Worker")

In [26]:
tom = Parent()

In [27]:
tom.job()

Worker


In [28]:
def new_job():
    print("Manager")

In [29]:
tom.job = new_job

In [30]:
tom.job()

Manager


## 协程

与子例程一样，协程也是一种程序组件。相对子例程而言，协程更为一般和灵活，它是在同一个线程中运行的。在Python中，yield可以实现协程。另外，还有很多第三方的版本，比如greenlet。

**进程、线程和协程之间的关系和区别如下：**
* 进程拥有自己独立的堆和栈，既不共享堆，亦不共享栈，进程由操作系统调度。
* 线程拥有自己独立的栈和共享的堆，共享堆，不共享栈，线程亦由操作系统调度。
* 协程和线程一样共享堆，不共享栈，协程由程序员在协程的代码里显示调度。